In [1]:
import os
import pandas as pd
import gzip
from pysam import VariantFile
from dataclasses import dataclass, field
from collections import defaultdict
from typing import Set, Dict, List
import random
from tqdm import tqdm
import numpy as np
import string
from pathlib import Path

In [2]:
random.seed(10)

In [3]:
working_dir = "./data/"
local_filename = "GMKF-OFC-GREGoR-denovo-SV-Master-20240918.txt"

In [4]:
@dataclass
class IndividualRelationship:
    family_id: str
    individual_id: str
    father_id: str
    mother_id: str
    gender: int
    affected: int


@dataclass
class Pedigree:
    fathers: Set[str] = field(default_factory=set)
    mothers: Set[str] = field(default_factory=set)
    relationships: Dict[str, IndividualRelationship] = field(default_factory=dict)
    updated_relationships: Dict[str, IndividualRelationship] = field(default_factory=dict)  # includes the original relationship and swapped parents when possible
    subject_variants: Dict[str, List[str]] = field(default_factory=dict)  # a per-sample list of inherited variants made synthetic de novo by swapped parents

    
@dataclass
class FilePair:
    vcf_filename: str
    ped_filename: str
    pedigree: Pedigree = field(default_factory=Pedigree)

In [5]:
def grf(filename):  # get resolved filename
    return os.path.join(working_dir, filename)

vcf_ped_pair_filenames = [
    FilePair(grf("mg_batch20.annotated.vcf.gz"), grf("mg_batch20_ware-20231120.VCFids.ped")),  # ware
    FilePair(grf("m4_batch03.annotated.vcf.gz"), grf("mg_batch12_krantz-20230922.VCFids.ped")),  # krantz
    FilePair(grf("m2_batch11.annotated.vcf.gz"), grf("mg_batch02_alba.ped.txt")),  # butali (not in the workspace)
    FilePair(grf("m2_batch10.annotated.vcf.gz"), grf("mg_batch01_ped.tsv")),  # beaty
    FilePair(grf("mg_batch13.annotated.vcf.gz"), grf("mg_batch13_alba.ped.txt")),  # leslie
    FilePair(grf("mg_batch03.annotated.vcf.gz"), grf("ped_mg_batch03.tsv")), # chung (not in the workspace)
    FilePair(grf("mg_batch05.annotated.vcf.gz"), grf("mg_batch05_ped.txt")), # engle (not in the workspace)
    FilePair(grf("mg_batch17.annotated.vcf.gz"), grf("mg_batch17_ped.tsv")), # marazita (not in the workspace)
    FilePair(grf("mg_batch08.annotated.vcf.gz"), grf("mg_batch08_gleeson-20230727.VCFids.ped")),  # gleeson
    FilePair(grf("mg_batch07.annotated.vcf.gz"), grf("ped_mg_batch07.tsv")),  # gharavi
    # FilePair("phase4_all_batches.annotated.vcf.gz", "..."),  # gregor
    # FilePair("second_run_cp_cohort.annotated.vcf.gz", "...")  # OFC
]

In [6]:
df = pd.read_csv(os.path.join(working_dir, local_filename), sep="\t")
df = df.reset_index() # to make sure indexes pair with the number of rows

/var/folders/6b/x9ksqf1d63j3bsvvls_04wfr0000gq/T/ipykernel_89777/3919829110.py:1: DtypeWarning: Columns (85,106,107,114,115,126,130,133,135,136,137,138) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(working_dir, local_filename), sep="\t")


In [7]:
# Drop rows with NaN in the result_final column
df = df[df["result_final"].notna()]

### Extract labeled true not de novo variants (inherited variants)

In [8]:
inherited_variants = df[df["result_final"] != "yes"]

In [9]:
print(f"All variants:       {df.shape[0]:,}")
print(f"Inherited variants: {inherited_variants.shape[0]:,}")

All variants:       19,575
Inherited variants: 16,337


### Data prepration: Extract parents in each PED file

In [10]:
ped_file_header = ""

for file_pair in vcf_ped_pair_filenames:
    ped_filename = file_pair.ped_filename
    with open(ped_filename, "r") as f:
        p = file_pair.pedigree
        ped_file_header = f.readline()
        for line in f:
            cols = line.strip().split("\t")
            indiv = cols[1]
            father = cols[2]
            mother = cols[3]

            p.fathers.add(father)
            p.mothers.add(mother)
            p.relationships[indiv] = IndividualRelationship(cols[0], indiv, father, mother, cols[4], cols[5])
        p.updated_relationships = p.relationships

### Reorganize the inherited variants

Assuming `sample` in the `df` refers to the individual ID, and we should be able to find a hit for that sample ID in only one of the PED files.

In [11]:
def get_pair(individual_id):
    for file_pair in vcf_ped_pair_filenames:
        if individual_id in file_pair.pedigree.relationships:
            return file_pair
    # print(f"Individual ID ({individual_id}) not found in the PED files.")  # TEMP 
    raise KeyError(f"Individual ID ({individual_id}) not found in the PED files.")

In [12]:
def get_samples_genotypes(variant, sample_ids):
    samples_genotype = {}
    for sample_id in sample_ids:
        sample = variant.samples.get(sample_id)
        if sample is None:
            samples_genotype[sample_id] = False
        else:
            samples_genotype[sample_id] = True if sample.get("GT") == (0, 0) else False
    
    # True: variant is not in the parent (i.e., de novo w.r.t that parent) and can be used for swapping
    # False: either the parent does not have a genotype for that variant, or the parent also has this variant, hence, cannot be used for the swapping
    return samples_genotype

In [13]:
def get_best_parent_swap(fathers_genotypes, mothers_genotypes, original_relationship: IndividualRelationship):
    permutations = defaultdict(list)
    for variant_id in fathers_genotypes.keys():  # iterate over genotypes of each variant for parent combinations
        for father_id, father_not_carries_variant in fathers_genotypes[variant_id].items():
            for mother_id, mother_not_carries_variant in mothers_genotypes[variant_id].items():
                if father_not_carries_variant and mother_not_carries_variant:
                    permutations[(father_id, mother_id)].append(variant_id)

    if len(permutations) == 0:
        # This happens when no combination of parents where neither is carring the variant is found. 
        return None

    max_potential_variants_count = -1
    for _, potential_variants in permutations.items():
        max_potential_variants_count = max(max_potential_variants_count, len(potential_variants))

    good_permutations = {x: variants for x, variants in permutations.items() if len(variants) == max_potential_variants_count}
    best_permutations = {x: variants for x, variants in good_permutations.items() if x[0] == original_relationship.father_id or x[1] == original_relationship.mother_id}

    # returning the first item from best matches, otherwise, the first items from the good matches.
    suggested_swap = next(iter(best_permutations.items())) if len(best_permutations) > 0 else next(iter(good_permutations.items()))

    return (
        IndividualRelationship(
            family_id= "synthetic_" + "".join(random.choices(string.ascii_lowercase + string.digits, k=10)),
            individual_id=original_relationship.individual_id, 
            father_id=suggested_swap[0][0], 
            mother_id=suggested_swap[0][1], 
            gender=original_relationship.gender, 
            affected=original_relationship.affected),
        suggested_swap[1])

In [14]:
test_max_samples_to_process = 20

In [15]:
test_processed_samples = 0

for sample, sample_variants_df in tqdm(
    inherited_variants.groupby("sample"), 
    total=inherited_variants['sample'].nunique(), 
    desc="Processing Samples"):

    try:
        file_pair = get_pair(sample)
    except:
        continue
    relationship = file_pair.pedigree.relationships[sample]

    fathers_genotypes = {}
    mothers_genotypes = {}

    vcf = VariantFile(file_pair.vcf_filename)
    for _, row in sample_variants_df.iterrows():
        for variant in vcf.fetch(row["chrom"], row["start"], row["end"]):
            individual = variant.samples.get(sample)
            if individual.get("GT") == (0, 0):
                # Discussed with Harrison. 
                # There are cases this could be a valid de novo, 
                # but since very rare, we're excluding them.
                continue

            original_father = variant.samples.get(relationship.father_id)
            original_mother = variant.samples.get(relationship.mother_id)
            if original_father is None:
                print(f"A sample with the father ID ({relationship.father_id}) not found in the VCF. Skipping this variant")
                continue
            if original_mother is None:
                print(f"A sample with the mother ID ({relationship.mother_id}) not found in the VCF. Skipping this variant")
                continue

            if original_father.get("GT") == (0, 0) and original_mother.get("GT") == (0, 0):
                # Same as above, discussed with Harrison. 
                # There are cases this could be a valid de novo, 
                # but since very rare, we're excluding them.
                continue

            fathers_genotypes[variant.id] = get_samples_genotypes(variant, file_pair.pedigree.fathers)
            mothers_genotypes[variant.id] = get_samples_genotypes(variant, file_pair.pedigree.mothers)

    suggested_swap = get_best_parent_swap(fathers_genotypes, mothers_genotypes, relationship)
    if suggested_swap is None:
        # i.e., no parent combination making at least one of the variants of sample a synthetic de novo is found.
        continue
    
    file_pair.pedigree.updated_relationships[sample] = suggested_swap[0]
    file_pair.pedigree.subject_variants[sample] = suggested_swap[1]

    test_processed_samples += 1
    if test_processed_samples >= test_max_samples_to_process:
        break


Processing Samples:   1%|          | 28/4248 [00:09<25:01,  2.81it/s]


In [16]:
assert \
    len(file_pair.pedigree.relationships) == len(file_pair.pedigree.updated_relationships), \
    "The number of items in the original and updated PED file do not match."

### Serialize updated PED files and list of subject variants

In [17]:
for file_pair in vcf_ped_pair_filenames:
    updated_ped_filename = os.path.join(working_dir, Path(file_pair.ped_filename).stem + "_updated.ped")
    subject_vars_filename = os.path.join(working_dir, Path(file_pair.ped_filename).stem + "_subject_variants_ids.tsv")

    with open(updated_ped_filename, "w") as f:
        f.write(ped_file_header.strip() + "\n")
        for _, r in file_pair.pedigree.updated_relationships.items():
            f.write("\t".join([r.family_id, r.individual_id, r.father_id, r.mother_id, r.gender, r.affected]) + "\n")
    print(f"Serialized updated relationships to {updated_ped_filename}")

    with open(subject_vars_filename, "w") as f:
        f.write("\t".join(["sample_id", "variants_ids"]) + "\n")
        for s_id, v_ids in file_pair.pedigree.subject_variants.items():
            f.write("\t".join([s_id, ";".join(v_ids)]) + "\n")
    print(f"{len(file_pair.pedigree.subject_variants)} subject variants are serialized to {subject_vars_filename}")

Serialized updated relationships to ./data/mg_batch20_ware-20231120.VCFids_updated.ped
0 subject variants are serialized to ./data/mg_batch20_ware-20231120.VCFids_subject_variants_ids.tsv
Serialized updated relationships to ./data/mg_batch12_krantz-20230922.VCFids_updated.ped
0 subject variants are serialized to ./data/mg_batch12_krantz-20230922.VCFids_subject_variants_ids.tsv
Serialized updated relationships to ./data/mg_batch02_alba.ped_updated.ped
0 subject variants are serialized to ./data/mg_batch02_alba.ped_subject_variants_ids.tsv
Serialized updated relationships to ./data/mg_batch01_ped_updated.ped
0 subject variants are serialized to ./data/mg_batch01_ped_subject_variants_ids.tsv
Serialized updated relationships to ./data/mg_batch13_alba.ped_updated.ped
0 subject variants are serialized to ./data/mg_batch13_alba.ped_subject_variants_ids.tsv
Serialized updated relationships to ./data/ped_mg_batch03_updated.ped
0 subject variants are serialized to ./data/ped_mg_batch03_subject_v